In [ ]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from google.colab.patches import cv2_imshow

In [ ]:
protoPath = os.path.sep.join("/content/drive/MyDrive/Myphotos/model/deploy.prototxt")
modelPath = os.path.sep.join("/content/drive/MyDrive/Myphotos/model/res10_300x300_ssd_iter_140000_fp16 (1).caffemodel")
detector = cv2.dnn.readNetFromCaffe("/content/drive/MyDrive/Myphotos/model/deploy.prototxt" , "/content/drive/MyDrive/Myphotos/model/res10_300x300_ssd_iter_140000_fp16 (1).caffemodel")
embedder = cv2.dnn.readNetFromTorch("/content/drive/MyDrive/Myphotos/model/openface_nn4.small2.v1.t7")

In [ ]:
imagePaths = list(paths.list_images("/content/drive/MyDrive/Myphotos/FaceDataset"))
print(imagePaths[:10])

['/content/drive/MyDrive/Myphotos/FaceDataset/00299.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00300.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00301.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00302.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00303.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00304.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00305.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00306.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00307.jpg', '/content/drive/MyDrive/Myphotos/FaceDataset/00308.jpg']


In [ ]:
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

In [ ]:
# initialize the total number of faces processed
total=0

Blob from Image(Binary Large Objects)

Audio, Video and Image data. Group of connected pixels in a binary image which shares a common property.

In the field of deep learning and classification, we preprocess the data in two ways:

Mean Subtraction

Scaling by some factor

Open cv's DNN consists of two functions for pre processing for classification

cv2.dnn.blobFromImages and cv2.dnn.blobFromImage

Steps in Pre Processing:

1.Decrease the average value

2.Zoom

3.Channel exchange

In [ ]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	if(image is None):
		pass
	else:
		
		image = imutils.resize(image, width=600)
		(h, w) = image.shape[:2]

		# construct a blob from the image
		imageBlob = cv2.dnn.blobFromImage(
			cv2.resize(image, (300, 300)), 1.0, (300, 300),
			(104.0, 177.0, 123.0), swapRB=False, crop=False)

		# apply OpenCV's deep learning-based face detector to localize
		# faces in the input image
		detector.setInput(imageBlob)
		detections = detector.forward()

		# ensure at least one face was found
		if len(detections) > 0:
			# we're making the assumption that each image has only ONE
			# face, so find the bounding box with the largest probability
			i = np.argmax(detections[0, 0, :, 2])
			confidence = detections[0, 0, i, 2]

			# ensure that the detection with the largest probability also
			# means our minimum probability test (thus helping filter out
			# weak detections)
			if confidence > 0.95:
				# compute the (x, y)-coordinates of the bounding box for
				# the face
				box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
				(startX, startY, endX, endY) = box.astype("int")

				# extract the face ROI and grab the ROI dimensions
				face = image[startY:endY, startX:endX]
				cv2_imshow(face)
				if(cv2.waitKey(1) and 0xFF==27):
					break
				(fH, fW) = face.shape[:2]

				# ensure the face width and height are sufficiently large
				if fW < 20 or fH < 20:
					continue

				# construct a blob for the face ROI, then pass the blob
				# through our face embedding model to obtain the 128-d
				# quantification of the face
				faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
					(96, 96), (0, 0, 0), swapRB=True, crop=False)
				embedder.setInput(faceBlob)
				vec = embedder.forward()

				# add the name of the person + corresponding face
				# embedding to their respective lists
				knownNames.append(name)
				knownEmbeddings.append(vec.flatten())
				total += 1

print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open("/content/drive/MyDrive/Myphotos/output/embeddings.pickle", "wb")
data.append(pickle.dumps(data))
f.close()